In [5]:
import pandas as pd
import numpy as np
data = pd.read_csv("master_dataset/merged_cleaned.csv")
data.head()


,title,text,subject,date,class,date_ofmonth,month,year,day_of_week,year_month,text_without_stopwords,title_without_stopwords
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,2017-12-31,1,31,12,2017,Sunday,2017-12,Donald Trump wish Americans Happy New Year lea...,Donald Trump Sends Out Embarrassing New Year’s...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,2017-12-31,1,31,12,2017,Sunday,2017-12,House Intelligence Committee Chairman Devin Nu...,Drunk Bragging Trump Staffer Started Russian C...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,2017-12-30,1,30,12,2017,Saturday,2017-12,"On Friday, revealed former Milwaukee Sheriff D...",Sheriff David Clarke Becomes An Internet Joke ...
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,2017-12-29,1,29,12,2017,Friday,2017-12,"On Christmas day, Donald Trump announced would...",Trump Is So Obsessed He Even Has Obama’s Name ...
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,2017-12-25,1,25,12,2017,Monday,2017-12,Pope Francis used annual Christmas Day message...,Pope Francis Just Called Out Donald Trump Duri...


In [6]:
#text-based features
#word count
import nltk
import string
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

def word_count(text):
    text = str(text).lower()
    text = text.replace("\r\n", ' ')
    if text == "no title":
        return 0
    else:
        return len(str(text).split(' '))

def sentence_count(text):
    sentences = nltk.sent_tokenize(text)
    return len(sentences)

def average_word_length(text):
    words = text.split()
    count = 0
    for word in words:
        count += len(word)
    if len(words) == 0:
        return 0
    else:
        return count / len(words)
        
def punctuation_count(text):
    count = 0
    for word in text:
        if word in string.punctuation:
            count += 1
    return count

def stopword_count(text):
    stopword = stopwords.words('english')
    count = 0
    for word in text.split():
        if word in stopword:
            count += 1
    return count

#change data type to string
data['text'] = data['text'].astype(str)
data['title'] = data['title'].astype(str)

data['text_word_count'] = data['text'].apply(word_count)
data['title_word_count'] = data['title'].apply(word_count)

data['text_sentence_count'] = data['text'].apply(sentence_count)
data['title_sentence_count'] = data['title'].apply(sentence_count)

data['text_average_word_length'] = data['text'].apply(average_word_length)
data['title_average_word_length'] = data['title'].apply(average_word_length)

data['text_punctuation_count'] = data['text'].apply(punctuation_count)
data['title_punctuation_count'] = data['title'].apply(punctuation_count)

data['text_stopwords_count'] = data['text'].apply(stopword_count)
data['title_stopwords_count'] = data['title'].apply(stopword_count)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
data

,title,text,subject,date,class,date_ofmonth,month,year,day_of_week,year_month,...,text_word_count,title_word_count,text_sentence_count,title_sentence_count,text_average_word_length,title_average_word_length,text_punctuation_count,title_punctuation_count,text_stopwords_count,title_stopwords_count
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,2017-12-31,1,31,12,2017,Sunday,2017-12,...,516,13,28,1,4.804040,5.583333,121,1,186,1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,2017-12-31,1,31,12,2017,Sunday,2017-12,...,309,9,11,1,5.213115,7.625000,39,0,119,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,2017-12-30,1,30,12,2017,Saturday,2017-12,...,600,16,25,1,5.168966,5.000000,148,0,209,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,2017-12-29,1,29,12,2017,Friday,2017-12,...,475,15,15,1,5.180180,4.571429,118,2,160,0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,2017-12-25,1,25,12,2017,Monday,2017-12,...,434,12,19,1,4.554762,5.363636,40,0,195,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44883,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,2017-08-22,0,22,8,2017,Tuesday,2017-08,...,485,9,15,1,5.015021,5.888889,71,4,179,1
44884,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,2017-08-22,0,22,8,2017,Tuesday,2017-08,...,134,7,6,1,5.336000,6.571429,18,0,45,1
44885,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,2017-08-22,0,22,8,2017,Tuesday,2017-08,...,337,7,16,1,5.043750,6.142857,48,0,127,2
44886,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,2017-08-22,0,22,8,2017,Tuesday,2017-08,...,213,9,8,1,4.814634,5.888889,21,0,81,2


In [8]:
# Write the newly cleaned dataframe in new file
data.to_csv("processed_data.csv", index=False) # Dataset with text feature